# Configuración de experimentos

En este notebook se resumen los experimentos llevados a cabo para la evaluación de los modelos seleccionados. El enfoque es el siguiente:

- Definición de pipelines con distintas combinaciones de componentes y modelos
- Selección de rangos de parámetros para los elementos del pipeline
- Ejecución de búsqueda aleatoria sobre la matriz de parámetros creada

Debido a la naturaleza de funcionamiento, se divide en dos secciones de experimentación: aprendizaje automático y aprendizaje profundo

In [ ]:
from pipelines import *
from globals.constants import *
from steps.input import tagged_audios

ImportError: attempted relative import with no known parent package

### Métricas de evaluación

Todos los experimentos utilizarán el siguiente conjunto de métricas para la validación de los clasificadores. Están orientadas a la robustez ante el desbalanceo de clases. Casos específicos pueden incluir adicionales:

- Efectividad (Accuracy) balanceada: extensión de la efetividad que evalúa las predicciones correctas por cada una de las clases
- F1: Balance entre la efectividad y la sensibilidad (Recall) del modelo para cada clase. El enfoque de agregación es el promedio de los F1 por clase
- Coeficiente de correlación de Matthews: Mide la correlación entre las clases previstas y su valor real

Se incluye también la métrica base usada por Sci-kit Learn, la efectividad estándar, para comparación.

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score, balanced_accuracy_score, matthews_corrcoef

metrics = {
    "f1": make_scorer(f1_score, average="macro"),
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "matthews_corrcoef": make_scorer(matthews_corrcoef),
    "accuracy": make_scorer(accuracy_score)
}

## Experimentos de Aprendizaje Automático

Este conjunto de experimentos está basado en tres modelos, dos simples y un ensemble:

- K-Vecinos más cercanos
- Máquina de Vectores Soporte
- Random Forest

Adicionalmente se seleccionaron dos métodos de ensemble para evaluar si mejoran los resultados del mejor de los modelos individuales:

- AdaBoost
- VotingClassifier


### Línea base

La línea base es un clasificador de K-Vecinos más cercanos con su configuración por defecto, sin emplear preprocesamiento a nivel de audio y utilizando la media de todas las features definidas. Los parámetros para las transformaciones a partir del dominio temporal al espectral también utilizan los parámetros por defecto. Debido a que es un método basado en distancias, sí se incluye un paso de escalado de variables. La técnica escogida es el escalado robusto, basado en medianas y rangos interquantílicos, dado que no se ha aplicado ningún tratamiento de outliers a los datos.

In [ ]:
baseline_config = {
    "steps": {
        FEATURE_EXTRACTOR: {
            "aggregator": ["mean"]
        },
        MODEL: {
            NAME: "KNeighborsClassifier"
        },
        FEATURE_TUNING: {
            NAME: "RobustScaler"
        }
    },    
    "model-type": "ml"
}

Se cargan los datos con los valores por defecto

In [ ]:
X, y = tagged_audios({})

 y se construye el pipeline utilizando validación cruzada

In [ ]:
from sklearn.model_selection import StratifiedKFold

base_pipeline = pipeline_factory(baseline_config)
base_results = cross_validate(base_pipeline, X, y, scoring=metrics, cv=StratifiedKFold(5), n_jobs=-1)

base_results

Puede observarse que el modelo base obtiene resultados algo mejores a un clasificador aleatorio en cuanto a la efectividad base. Sin embargo, las métricas robustas a desbalanceo de clases presentan resultados inferiores, lo que confirma que esta situación sesga las predicciones del modelo. En consecuencia, los diferentes experimentos emplearán una de las dos siguientes opciones para abordarlo:

- Configuración propia del modelo para casos que soporten clases desbalanceadas
- Técnicas de remuestreo

### Diseño de experimentos

Los experimentos a ejecutar son los siguientes:

- low-freq-knn:
    1. Modelo: K-Vecinos más cercanos 
    2. Detección de outliers: Rango IQR con enmascaramiento
    3. Reducción de ruido: Filtro de paso de banda para frequencias bajas
    4. Normalización: Estandarización de señales
    5. Extracción y selección de características: Media de todas features agregables y selección de K-mejores según la optimización de la distancia inter-intra clase
    6. Balanceo de clases: ADASYN
    7. Escalado de características: Escalado robusto
    8. Parámetros: Ver sección "optimize-params" de cada componente

- auto-knn:
    1. Modelo: K-Vecinos más cercanos 
    2. Detección de outliers: Rango IQR con corrección por interpolación cúbica
    3. Reducción de ruido: Filtro de Wiener
    4. Normalización: Estandarización de señales
    5. Extracción y selección de características: Media de todas features agregables y selección de K-mejores según la optimización de la distancia inter-intra clase
    6. Balanceo de clases: ADASYN
    7. Escalado de características: Escalado robusto
    8. Parámetros: Ver sección "optimize-params" de cada componente

- auto-rf:
    1. Modelo: Random Forest
    2. Detección de outliers: No
    3. Reducción de ruido: Filtro de Wiener
    4. Normalización: Nivelado de RMS
    5. Extracción y selección de características: Media de todas las features agregables y selección de K-mejores según la importancia dada por Random Forest
    6. Balanceo de clases: Nativo de Random Forest
    7. Escalado de características: No
    8. Parámetros: Ver sección "optimize-params" de cada componente

- adaptative-svm:
    1. Modelo: Máquina de vectores soporte
    2. Detección de outliers: Mixto de Z-score con detección de zonas muertas con enmascaramiento
    3. Reducción de ruido: Filtro de Wiener
    4. Normalización: Nivelado de RMS
    5. Extracción y selección de características: Media de todas las features agregables y selección de K-mejores por eliminación recursiva basada en el propio modelo
    6. Balanceo de clases: Nativo de Máquina de vectores soporte
    7. Escalado de características: Escalado robusto

- ada-tree:
    1. Modelo: AdaBoost con estimador por defecto (árbol de decisión)
    2. Detección de outliers: No
    3. Reducción de ruido: No
    4. Normalización: No
    5. Extracción y selección de características: Media de todas las features agregables y selección de K-mejores según la importancia dada por Random Forest
    6. Balanceo de clases: No
    7. Escalado de características: No

Los experimentos de búsqueda aleatoria realizarán un total de 30 iteraciones de entreamiento/evaluación con validación cruzada de 5 partes. El mejor modelo será escogido con respecto a la métrica F1.

El proceso implementa puntos de control al final de cada experimento para salvar resultados parciales en caso de fallo

In [ ]:
experiments_config = {
    "experiments": [
        {
            NAME: "low-freq-knn",
            "steps": {
                DETECTOR: {
                    NAME: "iqr"
                },
                FIXER: {
                    NAME: "mask"
                },
                DENOISER: {
                    NAME: "bandpass",
                    "optimize-params": {
                        "low": [10, 20, 50],
                        "high": [1000, 1100]
                    }
                },
                NORMALIZER: {
                    NAME: "standard"
                },
                FEATURE_EXTRACTOR: {
                    "aggregator": ["mean"]
                },
                FEATURE_SELECTOR: {
                    NAME: "kbest",
                    PARAMS: {
                        "method": "fisher_score"
                    },
                    "optimize-params": {
                        "k": [10, 15, 20]
                    }
                },
                FEATURE_TUNING: {
                    NAME: "RobustScaler"
                },
                MODEL: {
                    NAME: "KNeighborsClassifier",
                    "optimize-params": {
                        "class_balance": ["adasyn", "smote"],
                        "n_neighbors": [5, 7, 9, 11]
                    }
                }
            },
            METRICS: metrics,
            "model-type": "ml",
            "number-of-folds": 5,
            "search-parallel": 2,
            "mode": "search",
            "search-iters": 30
        },
        {
            NAME: "auto-knn",
            "steps": {
                DETECTOR: {
                    NAME: "iqr"
                },
                FIXER: {
                    NAME: "polyinterp",
                    PARAMS: {
                        "order": 3
                    }
                },
                DENOISER: {
                    NAME: "wiener",
                    "optimize-params": {
                        "mysize": [13, 15, 17, 19]
                    }
                },
                NORMALIZER: {
                    NAME: "standard"
                },
                FEATURE_EXTRACTOR: {
                    "aggregator": ["mean"]
                },
                FEATURE_SELECTOR: {
                    NAME: "kbest",
                    "optimize-params": {
                        "method": ["fisher_score", "mutual_info"],
                        "k": [10, 15, 20]
                    }
                },
                FEATURE_TUNING: {
                    NAME: "RobustScaler"
                },
                MODEL: {
                    NAME: "KNeighborsClassifier",
                    "optimize-params": {
                        "class_balance": ["adasyn", "smote"],
                        "n_neighbors": [5, 7, 9, 11]
                    }
                }
            },
            METRICS: metrics,
            "model-type": "ml",
            "number-of-folds": 5,
            "search-parallel": 2,
            "mode": "search",
            "search-iters": 30
        },
        {
            NAME: "auto-rf",
            "steps": {
                DENOISER: {
                    NAME: "wiener",
                    "optimize-params": {
                        "mysize": [13, 15, 17, 19]
                    }
                },
                NORMALIZER: {
                    NAME: "rms",
                    "optimize-params": {
                        "ref_rms": [0.05, 0.1, 0.2]
                    }
                },
                FEATURE_EXTRACTOR: {
                    "aggregator": ["mean"],
                    "exclude": ["rms"]
                },
                FEATURE_SELECTOR: {
                    NAME: "modelbased",
                    PARAMS: {
                        "method": "rf_importance"
                    },
                    "optimize-params": {
                        "k": [10, 15, 20]
                    }
                },
                MODEL: {
                    NAME: "RandomForestClassifier",
                    "optimize-params": {
                        "n_estimators": [50, 100, 200],
                        "max_depth": [3, 5, None]
                    },
                    "model_params": {
                            "class_weight": "balanced"
                    }
                }
            },
            METRICS: metrics,
            "model-type": "ml",
            "number-of-folds": 5,
            "search-parallel": 2,
            "mode": "search",
            "search-iters": 30
        },
        {
            NAME: "adaptative-svm",
            "steps": {
                DETECTOR: {
                    NAME: "twosided",
                    "optimize-params": {
                        "z_thresh": [3, 5],
                        "flat_frame_size": [512, 1024, 2048]
                    }
                },
                FIXER: {
                    NAME: "mask"
                },
                DENOISER: {
                    NAME: "wiener",
                    "optimize-params": {
                        "mysize": [13, 15, 17, 19]
                    }
                },
                NORMALIZER: {
                    NAME: "rms",
                    "optimize-params": {
                        "ref_rms": [0.05, 0.1, 0.2]
                    }
                },
                FEATURE_EXTRACTOR: {
                    "aggregator": ["mean"],
                    "exclude": ["rms"]
                },
                FEATURE_SELECTOR: {
                    NAME: "kbest",
                    PARAMS: {
                        "method": "mutual_info",
                        "k": 20
                    }
                },
                FEATURE_TUNING: {
                    NAME: "RobustScaler"
                },
                MODEL: {
                    NAME: "SVC",
                    "optimize-params": {
                        "kernel": ["linear", "rbf"],
                        "C": [0.1, 0.5, 1, 5, 10, 100],
                        "gamma": ["scale", "auto"]
                    },
                    "model_params": {
                        "class_weight": "balanced"
                    }
                }
            },
            METRICS: metrics,
            "model-type": "ml",
            "number-of-folds": 5,
            "search-parallel": 2,
            "mode": "search",
            "search-iters": 30
        },
        {
            NAME: "ada-tree",
            "steps": {
                FEATURE_EXTRACTOR: {
                    "aggregator": ["mean"]
                },
                FEATURE_SELECTOR: {
                    NAME: "modelbased",
                    PARAMS: {
                        "method": "rf_importance",
                        "k": 15
                    }
                },
                MODEL: {
                    NAME: COMPOSITE,
                    CONFIG: {
                        "method": "ada",
                    }
                }
            },
            METRICS: metrics,
            "model-type": "ml",
            "number-of-folds": 5,
            "search-parallel": 2,
            "mode": "cross-val"
        },
    ]
}

In [ ]:
experiment_results = multi_experiment_runner(X, y, experiments_config, save_on_experiment=False)

experiment_results